<a href="https://colab.research.google.com/github/habilg/text-basics/blob/master/02A_word_representation(embedding)_high_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word representation(embedding)

This a way to describe every word with features to find their anology. it is not a matter this describing features are meaningfull for human. 
There are some methods to achieve it. But generally two main aproachs are:


> static embeding  
> dynamic embeding

maybe you think if we collect all the world words in a very huge corpus, we could find the unique final vector for every vocabulary.In fact, it is not the case, since words relationship are specific for every context, so we expect that same word embeding vector become diffrent from one context to another context.

static embedings are representation of words which obtained from a very large corpus. like fasttext, GloVe. However, dynamic representation is obtained form the text.

## What is Keras soloution?
it provides an `embedding layer`, which is trainable. It means that this layer could initilized with static weights or it could find proper weights during the trainig. 
input for this layer must be a type of `tf.keras.layers.TextVectorization, tf.keras.layers.StringLookup, or tf.keras.layers.IntegerLookup`.

In [1]:
from tensorflow.keras.layers import Embedding

In [ ]:
e=Embedding()

## Using pretrained weights-GloVe:

embedding_initializer argument must be initialized by a embedding matrix of the training words

In [22]:
dataset = [
  "I write, erase, rewrite",
  "Erase again, and then",
  "A poppy blooms.",
]

In [23]:
import numpy as np


embedding_dim=50 #based on using the glove.6B.50d.txt
embeddings_index = {}
with open('/content/drive/MyDrive/glove.6B.50d.txt') as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs
print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [36]:
list(embeddings_index.items())[0]

('the',
 array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
        -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
         2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
         1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
        -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
        -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
         4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
         7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
        -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
         1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
       dtype=float32))

In [24]:
# producing embedding_matrix
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(output_mode="int")
text_vectorization.adapt(dataset)

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

In [25]:
word_index

{'': 0,
 '[UNK]': 1,
 'erase': 2,
 'write': 3,
 'then': 4,
 'rewrite': 5,
 'poppy': 6,
 'i': 7,
 'blooms': 8,
 'and': 9,
 'again': 10,
 'a': 11}

In [26]:
max_tokens=len(word_index)
max_tokens

12

In [31]:
embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

embedding_matrix[2] #embeding vector for word 'erase'

array([-0.14342999, -0.21572   ,  0.95021999, -0.060348  , -0.19272   ,
        0.59243   ,  0.22474   ,  0.34395999, -0.19253001,  0.41418999,
       -0.60180998,  0.15369999, -0.41172999, -0.46133   ,  0.34940001,
        0.56075001, -0.16425   , -1.12510002,  0.95339   ,  0.058167  ,
       -0.47325   , -0.55941999,  0.018379  , -0.81616998,  0.28639001,
       -0.80637002, -0.91731   ,  0.012369  ,  1.03820002, -0.80427003,
        1.00250006,  0.59998   , -0.30706999, -0.27456999, -0.54820001,
        0.48576   , -0.44499999, -0.16799   ,  0.67354   , -1.19289994,
       -0.29357001, -0.69257998,  0.025876  ,  0.27287999, -0.26589   ,
        0.78315002,  0.5589    ,  0.60016   ,  0.16568001, -0.90461999])

In [35]:
from tensorflow.keras.initializers import Constant

embedding_layer = Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

### Make trainable a pretrained initializer

## Training the Layer from the ground.

In [37]:
from tensorflow.keras.preprocessing.text import one_hot

voc_size=100
onehot_repr=[one_hot(words,voc_size)for words in dataset]
onehot_repr

[[22, 74, 29, 3], [29, 20, 38, 87], [75, 83, 49]]

In [38]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sent_length=8
encoded_sentences=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
encoded_sentences

array([[ 0,  0,  0,  0, 22, 74, 29,  3],
       [ 0,  0,  0,  0, 29, 20, 38, 87],
       [ 0,  0,  0,  0,  0, 75, 83, 49]], dtype=int32)

In [49]:
from tensorflow.keras.models import Sequential


embedding_dim=10
model=Sequential()
model.add(Embedding(voc_size,embedding_dim,input_length=sent_length,mask_zero=True))
model.compile('adam','mse')

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 8, 10)             1000      
                                                                 
Total params: 1,000
Trainable params: 1,000
Non-trainable params: 0
_________________________________________________________________


In [53]:
print(encoded_sentences[0])
print(model.predict(encoded_sentences[0]))

[ 0  0  0  0 22 74 29  3]
1/1 [==============================] - 0s 26ms/step
[[-0.00569334  0.00257286  0.03579393  0.03102187 -0.02277086 -0.02277479
  -0.00182947  0.01928984 -0.0045648  -0.04526095]
 [-0.00569334  0.00257286  0.03579393  0.03102187 -0.02277086 -0.02277479
  -0.00182947  0.01928984 -0.0045648  -0.04526095]
 [-0.00569334  0.00257286  0.03579393  0.03102187 -0.02277086 -0.02277479
  -0.00182947  0.01928984 -0.0045648  -0.04526095]
 [-0.00569334  0.00257286  0.03579393  0.03102187 -0.02277086 -0.02277479
  -0.00182947  0.01928984 -0.0045648  -0.04526095]
 [ 0.04262597  0.03559327  0.02676548 -0.01363174 -0.03188284  0.04452128
   0.00305692  0.02038933  0.04472685 -0.04297394]
 [-0.02634639  0.0303622   0.0400802   0.01257122  0.02991608 -0.02740448
  -0.02059689 -0.00821769 -0.02704332  0.01997993]
 [-0.01714382  0.04435327  0.01093065 -0.03152341  0.00158608  0.00651101
  -0.00473733 -0.02673093  0.00446523  0.04212156]
 [ 0.04971078 -0.01255681  0.04776806 -0.022964

In [54]:
print(encoded_sentences[0])
print(model.predict(encoded_sentences[0]))

[ 0  0  0  0 22 74 29  3]
1/1 [==============================] - 0s 22ms/step
[[-0.00569334  0.00257286  0.03579393  0.03102187 -0.02277086 -0.02277479
  -0.00182947  0.01928984 -0.0045648  -0.04526095]
 [-0.00569334  0.00257286  0.03579393  0.03102187 -0.02277086 -0.02277479
  -0.00182947  0.01928984 -0.0045648  -0.04526095]
 [-0.00569334  0.00257286  0.03579393  0.03102187 -0.02277086 -0.02277479
  -0.00182947  0.01928984 -0.0045648  -0.04526095]
 [-0.00569334  0.00257286  0.03579393  0.03102187 -0.02277086 -0.02277479
  -0.00182947  0.01928984 -0.0045648  -0.04526095]
 [ 0.04262597  0.03559327  0.02676548 -0.01363174 -0.03188284  0.04452128
   0.00305692  0.02038933  0.04472685 -0.04297394]
 [-0.02634639  0.0303622   0.0400802   0.01257122  0.02991608 -0.02740448
  -0.02059689 -0.00821769 -0.02704332  0.01997993]
 [-0.01714382  0.04435327  0.01093065 -0.03152341  0.00158608  0.00651101
  -0.00473733 -0.02673093  0.00446523  0.04212156]
 [ 0.04971078 -0.01255681  0.04776806 -0.022964